## Create dataframe with tag column

In [1]:
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_md")

In [2]:
reports = pd.read_csv("clean_report_text_sent_April.csv")

In [3]:
rpt_sentences = reports['report_text_sent']

In [7]:
import mysql.connector as mysqlCon
import mysql.connector

import pickle as pkl


def get_connection():
    conn = connect_umls('aasha', '24H=1day', 'localhost', 'umls')
    return conn


def connect_umls(uname, password, host, database):
    connection = mysqlCon.connect(user=uname, password=password,
                                  host=host, database=database)
    return connection


def close_connection(connection):
    connection.close()


def get_entity(string):
    """ get definitions/descriptions for the given string """
    try:
        # Get descritpions
        sql_select_Query = "select * from mrconso where STR like '" + string + "';"
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute(sql_select_Query)
        # get all records
        return cursor.fetchone()
    except mysql.connector.Error as e:
        print(e.msg)
        return ""
    finally:
        close_connection(conn)


def symantic_type_lookup(string):
    """ get definitions/descriptions for the given string """
    """ Get Semantic Typ for a given string """
    try:
        # Get descritpions
        # sql_select_Query = "select * from mrconso where STR like '" + string + "';"
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT sty FROM MRCONSO a, MRSTY b WHERE a.cui = b.cui AND str = %s; ", (string,))
        # get all records
        next_user = cursor.fetchone()
        if next_user is not None:
            user_blah = next_user[0]
        else:
            user_blah = ''
        return user_blah
    except mysql.connector.Error as e:
        print(e.msg)
        return ""
    finally:
        close_connection(conn)


def is_symantic_type_found(string):
    """ find whether any semantic type available for the given string """
    """ return True if semantic type is available otherwise return False """
    try:
        sem_type_list = symantic_type_lookup(string)
        if not sem_type_list:
            return False
        else:
            return True
    except Exception as e:
        print(e)
        return False


def cui_lookup(string):
    """ get definitions/descriptions for the given string """
    """ Get Semantic Typ for a given string """
    try:
        # Get descritpions
        # sql_select_Query = "select * from mrconso where STR like '" + string + "';"
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT cui FROM MRCONSO WHERE str = %s;", (string,))
        #row = cursor.fetchone()
        next_user = cursor.fetchone()
        if next_user is not None:
            user_blah = next_user[0]
        else:
            user_blah = ''
        return user_blah
    except mysql.connector.Error as e:
        print(e.msg)
        return ""
    finally:
        close_connection(conn)


def is_cui_found(string):
    """ find whether any semantic type available for the given string """
    """ return True if semantic type is available otherwise return False """
    try:
        sem_type_list = cui_lookup(string)
        if not sem_type_list:
            return False
        else:
            return True
    except Exception as e:
        print(e)
        return False


In [8]:
from tqdm import tqdm

def get_training_data_sentence(sentence, i):
    G1 = pd.DataFrame(columns = ['Sentence #', 'Word', 'POS', 'Tag'])
    
    doc = nlp(sentence)
    doc_len = len(doc) - 1
    
    for token in doc:
        current_word = token.text
              
        token_index = token.i
        
        #	Sentence #	Word	POS	Tag
        
        train_data = {'Sentence #':i,'Word':current_word, 'POS':token.pos_, 'Tag':''}
        
        G1 = G1.append(train_data,ignore_index=True)
    return G1

def get_training_data_corpus(report_sentences):
    G1 = pd.DataFrame(columns = ['Sentence #', 'Word', 'POS', 'Tag'])
    for i, each_sentence in enumerate(tqdm(report_sentences)):
        print(each_sentence, ' - ', i)
        G1 = G1.append(get_training_data_sentence(each_sentence, i),ignore_index=True)
    return G1

In [9]:
%%time
train_df = get_training_data_corpus(rpt_sentences)

  0%|                                                                                 | 3/9805 [00:00<08:16, 19.72it/s]

Record date Campbell Orthopedic Associates Madera Circle Omak GA Habib Valenzuela M.D. Valdez Harlan Jr  -  0
February Har is a year old pound gentleman who is referred for consultation by Dr. Harlan Oneil  -  1
About a week ago he slipped on the driveway at home and sustained an injury to his left ankle  -  2
He was seen at TriCity Hospital and was told he had a fracture  -  3
He was placed in an air splint and advised to be partial weight bearing and he is using a cane  -  4


  0%|                                                                                 | 9/9805 [00:00<07:08, 22.88it/s]

He is here for routine followup  -  5
Past medical history is notable for no ankle injuries previously  -  6
He has a history of diabetes and sleep apnea  -  7
He takes Prozac Cardizem Glucophage and Amaryl  -  8
He is also followed by Dr. Harold Nutter for an arrhythmia  -  9
He does not smoke  -  10
He drinks minimally  -  11
He is a set designer at Columbia Pictures  -  12


  0%|▏                                                                               | 18/9805 [00:00<05:56, 27.48it/s]

...

No increase in BPH sx  -  9790
No new skin lesions  -  9791
No cp. habits no smoking sober MEDICATIONS ATENOLOL  -  9792
MG Tablets PO QD Take as directed CAPTOPRIL MG Tablets PO BID Take as directed ASPIRIN  -  9793
ACETYLSALICYLIC ACID  -  9794
MG Tablets PO QD Take as directed PE  -  9795
afeb wgt carotids no bruits nml thyroid no nodes  -  9796


100%|█████████████████████████████████████████████████████████████████████████████▉| 9802/9805 [06:16<00:00, 31.39it/s]

clear lunds RRR no s or murmur abd obese no masses or HSM prominent varicosities on entire LR leg non tender  -  9797
no calf swelling  -  9798
Minimal pedal edema  -  9799
PROBLEMS Hypertension increase captopril to bid  -  9800
SP total hip replacement Shoemaker stable Diabetes mellitus counselled on wgt  -  9801
loss check hgbAC  -  9802
Eye exam OK earlier this year  -  9803


100%|██████████████████████████████████████████████████████████████████████████████| 9805/9805 [06:16<00:00, 26.01it/s]

Basal cell carcinoma no new ones visible Varicose veins reassured recommend against further surgery macular degeneration fu with optho  -  9804
Wall time: 6min 16s


In [13]:
train_df['Sty_avail'] = train_df['Word'].apply(lambda x: symantic_type_lookup(x))

train_df['cui_avail'] = train_df['Word'].apply(lambda x: cui_lookup(x))

UnboundLocalError: local variable 'conn' referenced before assignment

In [12]:
train_df.to_csv("rpt_words_clean_24April_01.csv")